In [3]:
%matplotlib inline

import json
import pandas as pd
import matplotlib.pyplot as plt

## Load the Twitter Trend Snapshots

Take the raw fata from dynamoDB dump, clean it up, and load it into a pandas data frame.

In [43]:
with open('output.json', 'rb') as f:
    trends = json.load(f)

In [7]:
# use this to inform cleanup process
trends["Items"][0]

{u'placeID': {u'N': u'2407517'},
 u'trend': {u'M': {u'date': {u'S': u'2016-03-03'},
   u'mappedName': {u'S': u'world book day'},
   u'placeName': {u'S': u'Fresno'},
   u'tweetName': {u'S': u'#WorldBookDay'},
   u'tweetVolume': {u'N': u'225775'},
   u'url': {u'S': u'http://twitter.com/search?q=%23WorldBookDay'}}},
 u'trendTime': {u'N': u'1457022783'}}

In [44]:
trends_clean = []

In [45]:
for trend in trends["Items"]:
    new = {}
    new["placeID"] = trend["placeID"]["N"]
    new["trendMapped"] = trend["trend"]["M"]["mappedName"]["S"]
    new["placeName"] = trend["trend"]["M"]["placeName"]["S"]
    new["trendName"] = trend["trend"]["M"]["tweetName"]["S"]
    new["trendVolume"] = trend["trend"]["M"]["tweetVolume"]["N"]
    new["searchUrl"] = trend["trend"]["M"]["url"]["S"]
    new["trendDate"] = trend["trend"]["M"]["date"]["S"]
    new["trendTime"] = trend["trendTime"]["N"]
    trends_clean.append(new)

In [46]:
# it worked
trends_clean[0]

{'placeID': u'2407517',
 'placeName': u'Fresno',
 'searchUrl': u'http://twitter.com/search?q=%23WorldBookDay',
 'trendDate': u'2016-03-03',
 'trendMapped': u'world book day',
 'trendName': u'#WorldBookDay',
 'trendTime': u'1457022783',
 'trendVolume': u'225775'}

In [47]:
df_trends = pd.DataFrame(trends_clean)

## Pandas Cleanup
Converting columns and shit

In [48]:
df_trends.trendTime = df_trends.trendTime.astype(int)
df_trends.trendVolume = df_trends.trendVolume.astype(int)
df_trends.index = pd.to_datetime(df_trends.trendTime, unit = 's')
df_trends = df_trends.drop("trendTime", axis = 1)

In [72]:
df_trends[df_trends["placeName"] == 'Austin'].groupby(["placeName", "trendName"])["trendVolume"].sum().sort_values(ascending=False)

placeName  trendName              
Austin     #InternationalWomensDay    42256033
           #GOPDebate                 15906327
           #DemDebate                 15101032
           #ALDUBMaineEvent           12680017
           #UFC196                     7084952
           #IWD2016                    4865321
           #TrumpRally                 4265247
           #ALDUBPaandarNiDudang       3964532
           #GIFparty                   2951654
           #ALDUBTheProposition        2923100
           MANANALO SI KATH            2239664
           #MemeHistory                2201516
           #ALDUBAngPagdalaw           2179506
           #DolceAmorePuSo             1694586
           Kendrick                    1625493
           #DangerousWomanTonight      1437080
           Romney                      1075061
           #WorldBookDay                783033
           Ciara                        580942
           Bankroll Fresh               540323
           Peyton Manning

In [84]:
df_trends[(df_trends.placeName == "Austin") & (df_trends.trendDate == "2016-03-12")]

,placeID,placeName,searchUrl,trendDate,trendMapped,trendName,trendVolume
trendTime,,,,,,,
2016-03-12 00:18:02,2357536,Austin,http://twitter.com/search?q=Ciara,2016-03-12,ciara,Ciara,180332
2016-03-12 01:19:07,2357536,Austin,http://twitter.com/search?q=Ciara,2016-03-12,ciara,Ciara,196724
2016-03-12 01:49:09,2357536,Austin,http://twitter.com/search?q=Ciara,2016-03-12,ciara,Ciara,203886
2016-03-12 03:33:01,2357536,Austin,http://twitter.com/search?q=%23TrumpRally,2016-03-12,trump rally,#TrumpRally,291580
2016-03-12 03:34:06,2357536,Austin,http://twitter.com/search?q=%23TrumpRally,2016-03-12,trump rally,#TrumpRally,291580
2016-03-12 04:48:02,2357536,Austin,http://twitter.com/search?q=%23TrumpRally,2016-03-12,trump rally,#TrumpRally,368615
2016-03-12 05:48:01,2357536,Austin,http://twitter.com/search?q=%23TrumpRally,2016-03-12,trump rally,#TrumpRally,408820
2016-03-12 07:36:19,2357536,Austin,http://twitter.com/search?q=%23ALDUBPaandarNiD...,2016-03-12,aldub paandar ni dudang,#ALDUBPaandarNiDudang,1186290
2016-03-12 10:33:03,2357536,Austin,http://twitter.com/search?q=%23ALDUBPaandarNiD...,2016-03-12,aldub paandar ni dudang,#ALDUBPaandarNiDudang,1365676


In [53]:
df_trends.to_json("cleaned_trends.json",orient = 'records')
